In [2]:
import pandas as pd

In [50]:
df = pd.read_csv('full_dataset.csv')

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

id_counts = df['id'].value_counts() #id_counts is a list of the ids that appear 7 times. There are 1460 apartments.

result = df[df['id'].isin(id_counts.index[id_counts == 7])].copy()

result.head() #result is the dataframe that has all the apartments that appear seven times

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price,date
28,209a6d2dea99216bdf83a968343cd3b3,szczecin,tenement,61.48,2.0,NaN,3.0,1900.0,53.491508,14.606985,...,condominium,brick,premium,no,no,no,no,yes,390000,2023-12-01
30,bac1869ad9c2c1ef444b0ba6d225761a,szczecin,tenement,77.71,3.0,3.0,3.0,1925.0,53.433019,14.569775,...,condominium,brick,low,no,no,no,no,yes,599000,2023-12-01
33,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,cooperative,NaN,NaN,no,no,yes,no,no,420000,2023-12-01
34,7ec1e260b3f3985259ec7626756775c0,szczecin,tenement,89.57,4.0,1.0,4.0,1930.0,53.432411,14.538777,...,condominium,brick,premium,no,no,NaN,no,no,810000,2023-12-01
62,4cc526db21afa9861c7ec26124a0951e,szczecin,blockOfFlats,77.00,3.0,NaN,2.0,1906.0,53.445241,14.548302,...,condominium,brick,premium,no,no,no,no,yes,660000,2023-12-01


In [60]:
#print(id_counts)
print(id_counts.index[id_counts == 7].shape)

(1460,)


In [51]:
count=0
list = id_counts.to_list()
for i in list:
    if i == 7:
        count = count + 1

print(count)

1460


In [52]:
result.shape

(10220, 29)

In [53]:
print(10220/1460) #Verifying the correct number of apartments in result

7.0


In [54]:
feature_columns = df.columns.difference(['price', 'date']) #features to compare to in results dataframe
print(feature_columns)

Index(['buildYear', 'buildingMaterial', 'centreDistance', 'city',
       'clinicDistance', 'collegeDistance', 'condition', 'floor', 'floorCount',
       'hasBalcony', 'hasElevator', 'hasParkingSpace', 'hasSecurity',
       'hasStorageRoom', 'id', 'kindergartenDistance', 'latitude', 'longitude',
       'ownership', 'pharmacyDistance', 'poiCount', 'postOfficeDistance',
       'restaurantDistance', 'rooms', 'schoolDistance', 'squareMeters',
       'type'],
      dtype='object')


In [55]:
def check_same_features(group):
    return (group[feature_columns].nunique() == 1).all()

filtered_apartments = result.groupby('id').filter(check_same_features)['id'].unique()

print(filtered_apartments)
filtered_apartments.shape #There are 147 apartments out of 1460 that have the exact same values for all the features except for date and price 

['bac1869ad9c2c1ef444b0ba6d225761a' 'fb10546822255a8bd8d52448ce0ae265'
 '504193ed8e736e687f64d7ba05449972' 'bde946f27b37a2dc9219075cb954f929'
 'c5c9f8850f6ed88e3009a8f251b4edce' 'f77c9cb1826c25bc189289d03635fb37'
 'b4024bae3e66c01955881dab66b1ac11' '73741308978f8307cb4b9ad4d95fcc00'
 'bea463ee71f6b7f314e3a315a6a74647' '030752d63a69565baf9dd3a638779662'
 'c03739ddb00be21349eda6e2282eaa1a' 'dbd54f4d27a44b802de090e27a0e3b4f'
 'd96ca644f596e4b9de45302ad67e4200' '41efcb82af6a97f702020ae2e62c69ae'
 '0b761e5a60709da4c695bc86e3dd369e' '8d38bd2a7d249981b7b32ae1026c0c84'
 'c871bba481968ff830ff35172ecd6948' '83b8966fd5e2a1d328b5d002833c28bf'
 '0cf00d1553b0d51dd2a2b7dc73a1356b' 'a88fd72c2675d53d1071ac88af26e339'
 '86730d03d2cf793a127448f411d66016' 'cd9aabcaafd663968d56f932abdd18e8'
 'be1dcecca11916adf725a4e700bc9545' 'afb09688a60cac2ba17799f3d18fb3ce'
 '383f97c93b6406139658cb35305e89d6' '42f90670b6f0ebd72b45268c1db76695'
 '6f67218757bfba03cf29aa5c48d6e2c8' '3ce59077604ccd1e43d4c829cf435fce'
 'a0dc

(147,)

In [43]:
test = pd.DataFrame({'id': [1, 2, 2, 1, 3], 'feat1': [1, 1, 2, 1, 3], 'feat2': [1, 1, 2, 1, 3], 'feat3': [1, 1, 2, 1, 3]})
test.head()



,id,feat1,feat2,feat3
0,1,1,1,1
1,2,1,1,1
2,2,2,2,2
3,1,1,1,1
4,3,3,3,3


In [48]:
def test_check_same_features(group):
    return (group[['id','feat1','feat2','feat3']].nunique() == 1).all()

filtered = test.groupby('id').filter(test_check_same_features)['id'].unique()

filtered
#filtered.shape

array([1, 3])

In [61]:
result_df = result[result['id'].isin(filtered_apartments)].copy() #result_df is the final dataframe with apartments that appear 7 times, are the same for all features except for date and price.

#result = df[df['id'].isin(id_counts.index[id_counts == 7])].copy()
#result_df.to_csv('dupes.csv', index=False)

result_df.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price,date
30,bac1869ad9c2c1ef444b0ba6d225761a,szczecin,tenement,77.71,3.0,3.0,3.0,1925.0,53.433019,14.569775,...,condominium,brick,low,no,no,no,no,yes,599000,2023-12-01
410,fb10546822255a8bd8d52448ce0ae265,szczecin,tenement,78.60,3.0,2.0,4.0,1929.0,53.432833,14.548187,...,condominium,brick,premium,no,no,no,no,yes,440000,2023-12-01
437,504193ed8e736e687f64d7ba05449972,szczecin,tenement,100.00,3.0,4.0,4.0,1930.0,53.436494,14.572026,...,condominium,brick,premium,yes,no,no,no,yes,739000,2023-12-01
438,bde946f27b37a2dc9219075cb954f929,szczecin,blockOfFlats,96.06,4.0,3.0,3.0,2009.0,53.465600,14.541539,...,condominium,brick,premium,no,yes,no,no,no,799000,2023-12-01
442,c5c9f8850f6ed88e3009a8f251b4edce,szczecin,tenement,42.57,2.0,1.0,1.0,1938.0,53.495690,14.584651,...,condominium,brick,premium,yes,no,no,no,yes,379000,2023-12-01


In [62]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1029 entries, 30 to 117249
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    1029 non-null   object        
 1   city                  1029 non-null   object        
 2   type                  1029 non-null   object        
 3   squareMeters          1029 non-null   float64       
 4   rooms                 1029 non-null   float64       
 5   floor                 1029 non-null   float64       
 6   floorCount            1029 non-null   float64       
 7   buildYear             1028 non-null   float64       
 8   latitude              1029 non-null   float64       
 9   longitude             1029 non-null   float64       
 10  centreDistance        1029 non-null   float64       
 11  poiCount              1029 non-null   float64       
 12  schoolDistance        1029 non-null   float64       
 13  clinicDistance      

In [63]:
result_df.shape

(1029, 29)

In [64]:
print(1029/147)

7.0


In [65]:
result_df.to_csv('Dataset_to_merge.csv', index=False)